In [1]:
import pandas
import mmengine
import torch
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.detr3d.custom_waymo_metric import CustomNuscMetric,CustomWaymoMetric, Waymo_Evaluator
from projects.detr3d.custom_nusc_metric_native import NuScenesEval_native
from projects.configs import debug
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend
from projects.configs.bevdet_baseline import debug as bevdet

In [2]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
import os.path as osp
import numpy as np
from projects.detr3d.detr3d import DETR3D
detr3d = DETR3D()
vis = visualizer_zlt(debug_name='',vis_count=10000)


In [ ]:
# eval_results = pandas.read_pickle('/home/zhenglt/mmdev11/detr3d-dev11/results/results_1684912557.9543872.pkl')
# test_results = pandas.read_pickle('results/results_1684913989.0714428.pkl')

# bs8 big data
# eval_results = pandas.read_pickle('results/results_1684919722.0415556.pkl')
# test_results = pandas.read_pickle('results/results_1684919969.5447073.pkl')
# bs8 small data
eval_results = pandas.read_pickle('results/results_1684921574.3698196.pkl')
test_results = pandas.read_pickle('results/results_1684921731.9052155.pkl')
single_gpu_test_results = pandas.read_pickle('results/results_1684922392.2481089.pkl')
single_gpu_test_bs8_results = pandas.read_pickle('results/results_1684922683.1213005.pkl')

In [ ]:
# print(single_gpu_test_bs8_results[0])
# print(test_results[0])
tmp = torch.tensor(single_gpu_test_bs8_results[0]['pred_instances_3d']['scores_3d']*10000,dtype=torch.int32)
# print(tmp)
# print(float(single_gpu_test_bs8_results[0]['pred_instances_3d']['scores_3d'][0:1]))
# print(float(test_results[0]['pred_instances_3d']['scores_3d'][0:1]))
for i in range(0,len(eval_results)):
    print((torch.tensor(single_gpu_test_bs8_results[i]['pred_instances_3d']['scores_3d']*1000,dtype=torch.int32) == torch.tensor(test_results[i]['pred_instances_3d']['scores_3d']*1000,dtype=torch.int32)))

In [ ]:
print(eval_results[1]['pred_instances_3d']['bboxes_3d'])
print(eval_results[1]['gt_instances']['bboxes_3d'])

In [ ]:
print(test_results[1]['pred_instances_3d']['bboxes_3d'])
print(test_results[1]['gt_instances']['bboxes_3d'])

In [ ]:
test_results[1]['sample_idx']

In [ ]:
eval_input = pandas.read_pickle('/home/zhenglt/mmdev11/detr3d-dev11/debug/bevdet/eval_time_input/00001.pkl')
test_input = pandas.read_pickle('/home/zhenglt/mmdev11/detr3d-dev11/debug/bevdet/test_time_input/00001.pkl')

In [ ]:
# eval_input['data_samples'] == test_input['data_samples'] # check and clear
# (eval_input['input_dict']['imgs'] == test_input['input_dict']['imgs']).all()
for i in range(2390):
    eval_input = pandas.read_pickle('/home/zhenglt/mmdev11/detr3d-dev11/debug/bevdet/eval_time_input/{:05d}.pkl'.format(i))
    test_input = pandas.read_pickle('/home/zhenglt/mmdev11/detr3d-dev11/debug/bevdet/test_time_input/{:05d}.pkl'.format(i))
    print((eval_input['input_dict']['imgs'] == test_input['input_dict']['imgs']).all())
    del(eval_input)
    del(test_input)

In [ ]:
checker = []
for i in range(len(test_results)):
    print(i)
    checker.append((eval_results[i]['pred_instances_3d']['scores_3d'] == test_results[i]['pred_instances_3d']['scores_3d']).all().reshape(1,1))
checker = torch.cat(checker)
checker.all()


In [ ]:
# /home/zhenglt/mmdev11/detr3d-dev11/results/0.33N_bevdet_nusc_metric_results.json
nusc_metric_file = '/home/zhenglt/mmdev11/detr3d-dev11/results/0.33N_bevdet_nusc_metric_results.pkl'
waymo_metric_file = '/home/zhenglt/mmdev11/detr3d-dev11/results/0.33N_bevdet_waymo_metric_results.pkl'
nusc_metric_results = pandas.read_pickle(nusc_metric_file)
waymo_metric_results = pandas.read_pickle(waymo_metric_file)

In [ ]:
checker = []
for i in range(len(nusc_metric_results)):
    checker.append((nusc_metric_results[i]['pred_instances_3d']['bboxes_3d'].tensor == waymo_metric_results[i]['pred_instances_3d']['bboxes_3d'].tensor).all().reshape(1,1))
checker = torch.cat(checker)
checker.all()


In [ ]:
cfg = dict(
    data_root=bevdet.nusc_root,
    ann_file=bevdet.nusc_root + 'nuscenes_infos_val.pkl',
    metric='bbox'
)
evaluator = CustomNuscMetric(**cfg)

In [ ]:
val_info = pandas.read_pickle('data/nus_v2/debug_val.pkl')
evaluator.dataset_meta = val_info['metainfo']
evaluator.compute_metrics(waymo_metric_results)

In [5]:
Weval = CustomWaymoMetric()
# Wfomatter = Waymo_Evaluator(Weval.classes, Weval.k2w_cls_map)
# obj = Wfomatter.to_waymo_obj(waymo_metric_results, path='results/0.33N_bevdet_waymo_metric_result.bin',ins_key='pred_instances_3d')
# obj = Wfomatter.to_waymo_obj(waymo_metric_results, path='results/0.33N_bevdet_waymo_metric_gt.bin',ins_key='gt_instances')

In [3]:
# n2w_fx100_results = pandas.read_pickle('results/results_1691551727.77455.pkl')
# n2w_fx100_results = pandas.read_pickle('results/results_1691553795.392958.pkl') #actually n2l
# n2w_fx100_results = pandas.read_pickle('results/results_1691555137.4741979.pkl') #actually n2all
n2w_fx100_results = pandas.read_pickle('results/results_1691590325.439879.pkl') # actually w2all
n2w_fx100_results[0].keys()

dict_keys(['city_name', 'dataset_name', 'pred_instances_3d', 'sample_idx', 'gt_instances'])

In [7]:
for result in n2w_fx100_results:
    for name in ['pred_instances_3d','gt_instances']:
        pred = result[name]
        predboxes = pred['bboxes_3d']
        predboxes.tensor[:,2] = 0 - predboxes.tensor[:,5]/2
Weval.compute_metrics(n2w_fx100_results)

argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2
argoverse2

/home/zhenglt/mmdev11/detr3d-dev11/projects/detr3d/custom_waymo_metric.py:225: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred_labels = torch.tensor(result[pred]['labels_3d'],dtype=torch.long)


[                          ] 299/13945, 1272.8 task/s, elapsed: 0s, ETA:    11s

KeyboardInterrupt: 

In [16]:
single_results = []
for result in n2w_fx100_results:
    if 'kitti' == result['dataset_name']:
        for name in ['pred_instances_3d','gt_instances']:
            pred = result[name]
            predboxes = pred['bboxes_3d']
            predboxes.tensor[:,2] = 0 - predboxes.tensor[:,5]/2
        single_results.append(result)
Weval.compute_metrics(single_results)

Converting to waymo format...
[>>>>                       ] 565/3769, 1279.0 task/s, elapsed: 0s, ETA:     3s

In [ ]:
bevdet_results = pandas.read_pickle('/home/zhenglt/mmdev11/detr3d-dev11/results/results_1684915616.2373161.pkl')
Weval.compute_metrics(bevdet_results)

In [ ]:
val_gt = pandas.read_pickle('data/nus_v2/nuscenes_infos_val.pkl')
detr3d_file = '/home/zhenglt/mmdev11/detr3d-dev11/results/0.33N_detr3d_waymo_metric_results.pkl'
detr3d_results = pandas.read_pickle(detr3d_file)
obj_detr3d = Wfomatter.to_waymo_obj(detr3d_results, path='results/0.33N_detr3d_waymo_metric_result.bin',ins_key='pred_instances_3d')

In [ ]:
checker = []
for i in range(len(nusc_metric_results)):
    checker.append((detr3d_results[0]['gt_instances']['bboxes_3d'].tensor == waymo_metric_results[0]['gt_instances']['bboxes_3d'].tensor).all().reshape(1,1))
checker = torch.cat(checker)
checker.all()


In [ ]:
from waymo_open_dataset import label_pb2
from waymo_open_dataset.protos import metrics_pb2
new_objs = metrics_pb2.Objects()
selfclasses=['Car', 'Pedestrian', 'Cyclist']
selfk2w_cls_map = {
    'Car': label_pb2.Label.TYPE_VEHICLE,
    'Pedestrian': label_pb2.Label.TYPE_PEDESTRIAN,
    'Sign': label_pb2.Label.TYPE_SIGN,
    'Cyclist': label_pb2.Label.TYPE_CYCLIST,
}
prog_bar = mmengine.ProgressBar(len(val_gt['data_list']))
for sample_idx,i in enumerate(val_gt['data_list']):
    for ins in i['instances']:
        if ins['bbox_label_3d'] == 0:
            clsss = 0
        elif ins['bbox_label_3d'] == 7:
            clsss = 1
        elif ins['bbox_label_3d'] == 5:
            clsss = 2
        else:
            continue
        class_name = selfclasses[clsss]
        (x, y, z, l, w, h, rot) = tuple(ins['bbox_3d'])
        # LiDARInstance3DBoxes.limit_yaw
        while rot < -np.pi:
            rot += 2 * np.pi
        while rot > np.pi:
            rot -= 2 * np.pi

        box = label_pb2.Label.Box()
        box.center_x, box.center_y, box.center_z = x, y, z
        box.length, box.width, box.height = l, w, h
        box.heading = rot

        o = metrics_pb2.Object()
        o.object.type = selfk2w_cls_map[class_name]
        o.context_name = str(sample_idx)
        o.frame_timestamp_micros = sample_idx
        o.object.camera_synced_box.CopyFrom(box)

        new_objs.objects.append(o)
    prog_bar.update()

val_gt['metainfo']

In [ ]:
open('results/0.33N_nusc_gt_objects.bin', 'wb').write(new_objs.SerializeToString())

In [ ]:
from mmdet3d.evaluation.metrics.waymo_let_metric import compute_waymo_let_metric
compute_waymo_let_metric('results/0.33N_nusc_gt_objects.bin','results/0.33N_bevdet_waymo_metric_result.bin')

In [ ]:
Neval = NuScenesEval_native()

In [ ]:
import pandas
import mmengine
import torch
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.detr3d.custom_waymo_metric import CustomNuscMetric,CustomWaymoMetric, Waymo_Evaluator
from projects.detr3d.custom_nusc_metric_native import NuScenesEval_native
from projects.configs import debug
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend
from projects.configs.bevdet_baseline import debug as bevdet
detr3d_file = '/home/zhenglt/mmdev11/detr3d-dev11/results/0.33N_detr3d_waymo_metric_results.pkl'
detr3d_results = pandas.read_pickle(detr3d_file)
Neval = NuScenesEval_native()
Neval.evaluate(detr3d_results)

In [1]:
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.configs.bevdet_goodmono_withaug.debug import nusc_train
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend
from mmdet3d.registry import DATASETS
import pandas
import mmengine
register_all_modules()
nusc = DATASETS.build(nusc_train)

08/09 15:58:14 - mmengine - INFO - ------------------------------
08/09 15:58:14 - mmengine - INFO - The length of the dataset: 26719
08/09 15:58:14 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 105060 |
| truck                | 18913  |
| construction_vehicle | 3504   |
| bus                  | 4189   |
| trailer              | 4970   |
| barrier              | 34572  |
| motorcycle           | 2226   |
| bicycle              | 2303   |
| pedestrian           | 51121  |
| traffic_cone         | 21195  |
+----------------------+--------+


In [36]:
nusc[1]['inputs']['img'].shape

torch.Size([1, 3, 444, 790])